In [1]:
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install ipython-sql prettytable

  Using cached sqlalchemy-1.3.9-cp313-cp313-win_amd64.whl
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.46
    Uninstalling SQLAlchemy-2.0.46:
      Successfully uninstalled SQLAlchemy-2.0.46


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.


  Using cached sqlalchemy-2.0.46-cp313-cp313-win_amd64.whl.metadata (9.8 kB)
Using cached sqlalchemy-2.0.46-cp313-cp313-win_amd64.whl (2.1 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [3]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [4]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

In [5]:
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [6]:
%sql sqlite:///my_data1.db

In [7]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [8]:
#DROP THE TABLE IF EXISTS

pd.read_sql_query('PRAGMA table_info(SPACEXTABLE);', con)

,cid,name,type,notnull,dflt_value,pk
0,0,Date,TEXT,0,None,0
1,1,Time (UTC),TEXT,0,None,0
2,2,Booster_Version,TEXT,0,None,0
3,3,Launch_Site,TEXT,0,None,0
4,4,Payload,TEXT,0,None,0
5,5,PAYLOAD_MASS__KG_,INT,0,None,0
6,6,Orbit,TEXT,0,None,0
7,7,Customer,TEXT,0,None,0
8,8,Mission_Outcome,TEXT,0,None,0
9,9,Landing_Outcome,TEXT,0,None,0


In [9]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [10]:
query = 'SELECT DISTINCT "Launch_Site" FROM SPACEXTABLE ORDER BY "Launch_Site";'
unique_launch_sites = pd.read_sql_query(query, con)
unique_launch_sites

,Launch_Site
0,CCAFS LC-40
1,CCAFS SLC-40
2,KSC LC-39A
3,VAFB SLC-4E


In [11]:
query = """
SELECT *
FROM SPACEXTABLE
WHERE "Launch_Site" LIKE 'CCA%'
LIMIT 5;
"""
pd.read_sql_query(query, con)

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [12]:
query = """
SELECT SUM("Payload_Mass__kg_") AS total_payload_mass_kg
FROM SPACEXTABLE
WHERE "Customer" = 'NASA (CRS)';
"""
pd.read_sql_query(query, con)

,total_payload_mass_kg
0,45596


In [13]:
query = """
SELECT AVG("Payload_Mass__kg_") AS avg_payload_mass_kg
FROM SPACEXTABLE
WHERE "Booster_Version" = 'F9 v1.1';
"""
pd.read_sql_query(query, con)

,avg_payload_mass_kg
0,2928.4


In [14]:
query = """
SELECT MIN("Date") AS first_success_ground_pad_date
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Success (ground pad)';
"""
pd.read_sql_query(query, con)

,first_success_ground_pad_date
0,2015-12-22


In [15]:
query = """
SELECT DISTINCT "Booster_Version"
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Success (drone ship)'
  AND "Payload_Mass__kg_" > 4000
  AND "Payload_Mass__kg_" < 6000
ORDER BY "Booster_Version";
"""
pd.read_sql_query(query, con)

,Booster_Version
0,F9 FT B1021.2
1,F9 FT B1031.2
2,F9 FT B1022
3,F9 FT B1026


In [16]:
query = """
SELECT "Mission_Outcome", COUNT(*) AS total
FROM SPACEXTABLE
GROUP BY "Mission_Outcome";
"""
pd.read_sql_query(query, con)

,Mission_Outcome,total
0,Failure (in flight),1
1,Success,98
2,Success,1
3,Success (payload status unclear),1


In [17]:
query = """
SELECT DISTINCT "Booster_Version"
FROM SPACEXTABLE
WHERE "Payload_Mass__kg_" = (
    SELECT MAX("Payload_Mass__kg_")
    FROM SPACEXTABLE
);
"""
pd.read_sql_query(query, con)

,Booster_Version
0,F9 B5 B1048.4
1,F9 B5 B1049.4
2,F9 B5 B1051.3
3,F9 B5 B1056.4
4,F9 B5 B1048.5
5,F9 B5 B1051.4
6,F9 B5 B1049.5
7,F9 B5 B1060.2
8,F9 B5 B1058.3
9,F9 B5 B1051.6


In [18]:
query = """
SELECT
  substr("Date", 6, 2) AS "Month",
  "Landing_Outcome",
  "Booster_Version",
  "Launch_Site"
FROM SPACEXTABLE
WHERE substr("Date", 0, 5) = '2015'
  AND "Landing_Outcome" LIKE '%Failure (drone ship)%'
ORDER BY "Month", "Date";
"""
pd.read_sql_query(query, con)

,Month,Landing_Outcome,Booster_Version,Launch_Site
0,01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
1,04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40
